In [1]:
import os
import pickle
import pandas as pd

In [2]:
os.getcwd()

'/home/jonasmmiguel/Documents/learning/poli/thesis/wind-stf/prototyping'

# define CV splits

In [3]:
df_infer = capacity_factors_daily_2000to2015 = pd.read_hdf(
    path_or_buf='../data/05_model_input/df_infer.hdf', 
    key='df_infer'
)

df_infer.head(3)

,DE145,DE114,DE146,DE132,DE12A,DE133,DE12C,DE11C,DE118,DE119,...,DEG01,DEG0F,DE275,DE21C,DE234,DE251,DE276,DE278,DE718,DE943
date,,,,,,,,,,,,,,,,,,,,,
2013-01-01,0.219507,0.269458,0.240540,0.263019,0.593148,0.331149,0.555176,0.284140,0.357407,0.410291,...,0.300818,0.363388,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2013-01-02,0.083655,0.063571,0.043303,0.005984,0.080731,0.007986,0.066120,0.083709,0.078740,0.108224,...,0.150699,0.133930,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2013-01-03,0.246707,0.229298,0.337413,0.047907,0.279092,0.136004,0.237430,0.320095,0.460150,0.516133,...,0.348737,0.297686,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
cv_params = {
    'method': 'expanding_windows',
    'relsize_shortest_train_window': 0.6,
     # 'train_window_relsize_last_pass': 1.0,  # always considered to be 1.0 
    'n_passes': 3,
    'steps_ahead': 1,
    'size_forecast_window': 7,
}

In [5]:
cv_method = cv_params['method']
n_passes = cv_params['n_passes']
relsize_shortest_train_window = cv_params['relsize_shortest_train_window']
size_forecasting_window = cv_params['size_forecast_window']
steps_ahead = cv_params['steps_ahead']

In [6]:
len(df_infer) 

903

In [8]:
import numpy as np

In [9]:
cv_splits = {}

# max train size so that train  + gap (steps_ahead) + val still fit in inference data
relsize_longest_train_window = 1 - ((steps_ahead - 1) + size_forecasting_window) / len(df_infer)

window_relsize = np.linspace(
    start=relsize_shortest_train_window, 
    stop=relsize_longest_train_window, 
    num=n_passes
)

for p in range(n_passes):
    pass_id = 'pass ' + str(p + 1)
    
    train_end_idx = round(window_relsize[p] * len(df_infer))    

    cv_splits[pass_id] = {
        'train': slice(0, train_end_idx),
        'val': slice(train_end_idx, train_end_idx + size_forecasting_window)
    }  

In [10]:
cv_splits

{'pass 1': {'train': slice(0, 542, None), 'val': slice(542, 549, None)},
 'pass 2': {'train': slice(0, 719, None), 'val': slice(719, 726, None)},
 'pass 3': {'train': slice(0, 896, None), 'val': slice(896, 903, None)}}

In [12]:
df_infer[ cv_splits['pass 3']['val'] ].head()

,DE145,DE114,DE146,DE132,DE12A,DE133,DE12C,DE11C,DE118,DE119,...,DEG01,DEG0F,DE275,DE21C,DE234,DE251,DE276,DE278,DE718,DE943
date,,,,,,,,,,,,,,,,,,,,,
2015-06-16,0.067821,0.042093,0.041279,0.024269,0.062996,0.047092,0.068612,0.050378,0.065940,0.134152,...,0.031529,0.030162,0.092073,0.030687,0.130938,0.132674,0.135495,0.144597,0.261322,0.077474
2015-06-17,0.057019,0.024349,0.028549,0.016155,0.024480,0.025088,0.034544,0.045504,0.039359,0.085863,...,0.007432,0.009049,0.165047,0.073389,0.122293,0.096906,0.145408,0.161790,0.075481,0.157481
2015-06-18,0.215926,0.163687,0.165895,0.064575,0.125823,0.076412,0.106456,0.193386,0.091288,0.177450,...,0.111204,0.111379,0.600980,0.460003,0.402093,0.325368,0.544583,0.569123,0.521823,0.197626
2015-06-19,0.201746,0.107546,0.039876,0.027187,0.098378,0.023175,0.081513,0.153659,0.036200,0.092505,...,0.085729,0.106492,0.215049,0.140360,0.252018,0.207540,0.240341,0.261635,0.265724,0.220457
2015-06-20,0.101521,0.066823,0.029024,0.011947,0.040524,0.008972,0.037997,0.090460,0.034260,0.071043,...,0.082149,0.089142,0.243301,0.136113,0.178904,0.171069,0.178985,0.185638,0.155400,0.137370


In [14]:
targets = ['DEF0C', 'DE111']

In [17]:
df_infer[ cv_splits['pass 3']['val'] ][targets]

,DEF0C,DE111
date,,
2015-06-16,0.145396,0.046773
2015-06-17,0.338567,0.013631
2015-06-18,0.355809,0.083842
2015-06-19,0.370678,0.063776
2015-06-20,0.218804,0.031796
2015-06-21,0.048347,0.031256
2015-06-22,0.076476,0.092497
